In [35]:
import json
import pprint

import numpy as np
import numpy.linalg as la

# File IO
import os
from os.path import join
import glob
import pickle
from pathlib import Path

import cv2
%matplotlib inline
import matplotlib.pylab as pt

import math
import time


DEBUG = False


# Tranformation helper Functions

### Getter function for axes and origin of a sensors coordinate system

`Note: view is a field extracted from the config of sensors.`

For example, `view = config['cameras']['front_left']['view']`

In [36]:
def get_axes_of_a_view(view):
    """
        Extract the normalized axes of a sensor in the vehicle coordinate system
        
            view: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
    """
    x_axis = view['x-axis']
    y_axis = view['y-axis']
     
    x_axis_norm = la.norm(x_axis)
    y_axis_norm = la.norm(y_axis)
    
    if (x_axis_norm < EPSILON or y_axis_norm < EPSILON):
        raise ValueError("Norm of input vector(s) too small.")
        
    # normalize the axes
    x_axis = x_axis / x_axis_norm
    y_axis = y_axis / y_axis_norm
    
    # make a new y-axis which lies in the original x-y plane, but is orthogonal to x-axis
    y_axis = y_axis - x_axis * np.dot(y_axis, x_axis)
 
    # create orthogonal z-axis
    z_axis = np.cross(x_axis, y_axis)
    
    # calculate and check y-axis and z-axis norms
    y_axis_norm = la.norm(y_axis)
    z_axis_norm = la.norm(z_axis)
    
    if (y_axis_norm < EPSILON) or (z_axis_norm < EPSILON):
        raise ValueError("Norm of view axis vector(s) too small.")
        
    # make x/y/z-axes orthonormal
    y_axis = y_axis / y_axis_norm
    z_axis = z_axis / z_axis_norm
    
    return x_axis, y_axis, z_axis

def get_origin_of_a_view(view):
    """
        Extract the origin of a sensor configuration in the vehicle coordinate system
        
            view: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
    """
    return view['origin']

### Getter functions for Coordinate tranformation matrix: 
$$
\begin{bmatrix}
    R & T \\ 0 & 1
\end{bmatrix}
$$

In [37]:
def get_transform_to_global(view):
    """
        Get the Tranformation matrix to convert sensor coordinates to global coordinates
        from the view object of a sensor
        
            view: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
    """
    # get axes
    x_axis, y_axis, z_axis = get_axes_of_a_view(view)
    
    # get origin 
    origin = get_origin_of_a_view(view)
    transform_to_global = np.eye(4)
    
    # rotation
    transform_to_global[0:3, 0] = x_axis
    transform_to_global[0:3, 1] = y_axis
    transform_to_global[0:3, 2] = z_axis
    
    # origin
    transform_to_global[0:3, 3] = origin
    
    return transform_to_global

In [38]:
def get_transform_from_global(view):
    """
        Get the Tranformation matrix to convert global coordinates to sensor coordinates 
        from the view object of a sensor
        
            view: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
    """
    # get transform to global
    transform_to_global = get_transform_to_global(view)
    trans = np.eye(4)
    rot = np.transpose(transform_to_global[0:3, 0:3])
    trans[0:3, 0:3] = rot
    trans[0:3, 3] = np.dot(rot, -transform_to_global[0:3, 3])

    return trans

In [39]:
def transform_from_to(src, target):
    """
        Get the Tranformation matrix to convert from source sensor view to target sensor view
        
            src: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
            target: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
    """
    transform = np.dot(get_transform_from_global(target), \
                       get_transform_to_global(src))
    
    return transform

### Getter Functions for Rotation Matrix 
$$R_{3x3}$$

In [40]:
def get_rot_from_global(view):
    """
        Get the only the Rotation matrix to rotate sensor coordinates to global coordinates
        from the view object of a sensor
        
            view: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
    """
    # get transform to global
    transform_to_global = get_transform_to_global(view)
    # get rotation
    rot =  np.transpose(transform_to_global[0:3, 0:3])
    
    return rot

def get_rot_to_global(view):
    """
        Get only the Rotation matrix to rotate global coordinates to sensor coordinates 
        from the view object of a sensor
        
            view: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
    """
    # get transform to global
    transform_to_global = get_transform_to_global(view)
    # get rotation
    rot = transform_to_global[0:3, 0:3]
    
    return rot

def rot_from_to(src, target):
    """
        Get only the rotation matrix to rotate from source sensor view to target sensor view
        
            src: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
            target: 'view object'
                is a dictionary of the x-axis, y-axis and origin of a sensor
    """
    rot = np.dot(get_rot_from_global(target), get_rot_to_global(src))
    
    return rot

# Helper Functions for (image/Lidar/label) file names

In [41]:
def extract_sensor_file_name(file_name, root_path, sensor_name, ext):
    file_name_split = file_name.split('/')
    
    seq_name = file_name_split[-4]
    data_viewpoint = file_name_split[-2]
    
    file_name_sensor = file_name_split[-1].split('.')[0]
    file_name_sensor = file_name_sensor.split('_')
    file_name_sensor = file_name_sensor[0] + '_' + \
                        sensor_name + '_' + \
                        file_name_sensor[2] + '_' + \
                        file_name_sensor[3] + '.' + ext
    file_path_sensor = join(root_path, seq_name, sensor_name, data_viewpoint, file_name_sensor)

    return file_path_sensor

In [42]:
def extract_image_file_name_from_any_file_name(file_name, root_path):
    return extract_sensor_file_name(file_name, root_path, 'camera', 'png')

In [43]:
def extract_semantic_file_name_from_any_file_name(file_name, root_path):
    return extract_sensor_file_name(file_name, root_path, 'label', 'png')

In [44]:
def get_prev_directory(file_name):
    file_name_split = file_name.split('/')
    it = -1
    if not file_name_split[it]:
        it = it - 1
    return file_name.replace(file_name_split[it], '')

In [45]:
def create_unique_dir(dir_name):
    if dir_name[-1] == '/':
        try:
            os.mkdir(dir_name)
            if DEBUG:
                print(f'New directory created: {dir_name}')
        except FileExistsError :
            if DEBUG:
                print(f'{dir_name} Already Exists. Directory creation skipped')
    else:
        if DEBUG:
            print(f'ERROR: {dir_name} is not a Valid Directory')
            

In [46]:
def get_cam_name_from_file_name(file_name):
    file_name_array = file_name.split('/')
    view_point = file_name_array[-2]
    view_point_array = view_point.split('_')
    cam_name = view_point_array[-2] + '_' + view_point_array[-1]
    
    return cam_name

# Helper Functions for Images

In [47]:
def get_cv2_image(file_name_image, color_transform):
    # Create Image object and correct image color
    image = cv2.imread(file_name_image)
    image = cv2.cvtColor(image, color_transform)
    
    return image

In [48]:
def get_undistorted_cv2_image(file_name_image, config, color_transform):
    
    # Create Image object and correct image color
    image = get_cv2_image(file_name_image, color_transform)
    
    # Extract cam_name
    cam_name = get_cam_name_from_file_name(file_name_image)
    
    if cam_name in ['front_left', 'front_center', \
                    'front_right', 'side_left', \
                    'side_right', 'rear_center']:
        # get parameters from config file
        intr_mat_undist = \
                  np.asarray(config['cameras'][cam_name]['CamMatrix'])
        intr_mat_dist = \
                  np.asarray(config['cameras'][cam_name]['CamMatrixOriginal'])
        dist_parms = \
                  np.asarray(config['cameras'][cam_name]['Distortion'])
        lens = config['cameras'][cam_name]['Lens']
        
        if (lens == 'Fisheye'):
            return cv2.fisheye.undistortImage(image, intr_mat_dist,\
                                      D=dist_parms, Knew=intr_mat_undist)
        elif (lens == 'Telecam'):
            return cv2.undistort(image, intr_mat_dist, \
                      distCoeffs=dist_parms, newCameraMatrix=intr_mat_undist)
        else:
            return image
    else:
        print("Invalid camera name. Returning original image")
        return image

In [49]:
def hsv_to_rgb(h, s, v):
    """
        Colour format conversion from Hue Saturation Value to RGB.
    """
    if s == 0.0:
        return v, v, v
    
    i = int(h * 6.0)
    f = (h * 6.0) - i
    p = v * (1.0 - s)
    q = v * (1.0 - s * f)
    t = v * (1.0 - s * (1.0 - f))
    i = i % 6
    
    if i == 0:
        return v, t, p
    if i == 1:
        return q, v, p
    if i == 2:
        return p, v, t
    if i == 3:
        return p, q, v
    if i == 4:
        return t, p, v
    if i == 5:
        return v, p, q

In [50]:
def normalize_vector(vector, lb, ub):
    minimum = np.min(vector)
    maximum = np.max(vector)
    
    return lb + (ub - lb)*(vector - minimum)/(maximum-minimum)

# LIDAR Helper Function

## Using LIDAR data

- LiDAR data is provided in a camera reference frame.
- `np.load(file_name_lidar)` loads the LIDAR points dictionary
- LIDAR info
    - azimuth: 
    - row: y axis image location of the lidar point
    - lidar_id: id of the LIDAR that the point belongs to
    - depth: Point Depth
    - reflectance: 
    - col: x axis image location of the lidar point
    - points: 
    - timestamp: 
    - distance: 

LIDAR dictionary loading Example: 
```
root_path = './camera_lidar_semantic/'

# get the list of files in lidar directory
file_names = sorted(glob.glob(join(root_path, '*/lidar/*/*.npz')))

# read the lidar data
lidar_front_center = np.load(file_names[0])
```

In [51]:
def get_lidar_on_image(file_name_lidar, config, root_path, pixel_size=3, pixel_opacity=1):
    file_name_image = extract_image_file_name_from_any_file_name(file_name_lidar, root_path)
    image = get_undistorted_cv2_image(file_name_image, config, cv2.COLOR_BGR2RGB)
    
    lidar = np.load(file_name_lidar)
    
    # get rows and cols
    rows = (lidar['row'] + 0.5).astype(np.int)
    cols = (lidar['col'] + 0.5).astype(np.int)
  
    # lowest distance values to be accounted for in colour code
    MIN_DISTANCE = np.min(lidar['distance'])
    # largest distance values to be accounted for in colour code
    MAX_DISTANCE = np.max(lidar['distance'])

    # get distances
    distances = lidar['distance']  
    # determine point colours from distance
    colours = (distances - MIN_DISTANCE) / (MAX_DISTANCE - MIN_DISTANCE)
    colours = np.asarray([np.asarray(hsv_to_rgb(0.75 * c, \
                        np.sqrt(pixel_opacity), 1.0)) for c in colours])
    pixel_rowoffs = np.indices([pixel_size, pixel_size])[0] - pixel_size // 2
    pixel_coloffs = np.indices([pixel_size, pixel_size])[1] - pixel_size // 2
    canvas_rows = image.shape[0]
    canvas_cols = image.shape[1]
    for i in range(len(rows)):
        pixel_rows = np.clip(rows[i] + pixel_rowoffs, 0, canvas_rows - 1)
        pixel_cols = np.clip(cols[i] + pixel_coloffs, 0, canvas_cols - 1)
        image[pixel_rows, pixel_cols, :] = \
                (1. - pixel_opacity) * \
                np.multiply(image[pixel_rows, pixel_cols, :], \
                colours[i]) + pixel_opacity * 255 * colours[i]
    return image.astype(np.uint8), lidar

# MAIN

In [52]:
# # Pick a random LIDAR file from the custom data set
# np.random.seed()
# idx = np.random.randint(0, len(custom_lidar_files)-1)
# file_name_lidar = custom_lidar_files[idx]

# # Visualize LIDAR on image
# lidar_on_image, lidar  = get_lidar_on_image(file_name_lidar, config, root_path)
# pt.fig = pt.figure(figsize=(15, 15))
# pt.title('number of points are '+ str(len(lidar['row'])) )
# pt.imshow(lidar_on_image)
# pt.axis('off')

In [53]:
# # Visualize Semantic Image
# label_image = get_undistorted_cv2_image(extract_semantic_file_name_from_any_file_name(file_name_lidar, root_path) ,\
#                                         config, cv2.COLOR_BGR2RGB)

# pt.fig = pt.figure(figsize=(15, 15))
# pt.imshow(label_image)
# pt.axis('off')

### LIDAR data loading

In [54]:
# Open Config File
with open ('cams_lidars.json', 'r') as f:
    config = json.load(f)
    
# pprint.pprint(config)

In [55]:
# Create Root Path
root_path = '/hdd/a2d2-data/camera_lidar_semantic/'

In [56]:
# Count Number of LIDAR points in each file
def get_num_lidar_pts_list(file_names_lidar):
    num_lidar_points = []
    start = time.time()
    for file_lidar in file_names_lidar:
        n_points = len(np.load(file_lidar)['points'])
        num_lidar_points.append(n_points)
    end = time.time() - start
    return num_lidar_points


In [57]:
# Create a histogram
def create_hist_pts(points, xlabel='number of points', ylabel='freq', title='Histogram of points'):
    fig = pt.hist(points, 1000)
    pt.xlabel(xlabel)
    pt.ylabel(ylabel)
    pt.title(title)
    pt.show()
    return fig

In [58]:
# Save this list in a file
def save_list_to_pfile(list_, file_name='file.pkl'):
    with open(file_name, 'wb') as filehandle:
        pickle.dump(list_, filehandle)

In [59]:
# Load Lidar data
N = 10000
lidar_file_list = root_path + f'../dataset/lidar_files_{N}.pkl'

if Path(lidar_file_list).is_file():
    with open(lidar_file_list, 'rb') as handle:
        file_names_lidar = pickle.load(handle)
else:
    # Get the list of files in lidar directory
    lidar_dirs = '*/lidar/*/*.npz'                     # ALL LIDAR
    # lidar_dirs = '20180925_124435/lidar/*/*.npz'       # 1 - Front and Sides
    # lidar_dirs = '*/lidar/cam_front_center/*.npz'      # ALL front center
    file_names_lidar = sorted(glob.glob(join(root_path, lidar_dirs)))
    
    # Extract Lidar files with minimum N points
    num_lidar_points_list = get_num_lidar_pts_list(file_names_lidar)

    # Create Histogram
    create_hist_pts(num_lidar_points_list, title='Histogram of Lidar data-points')
    file_names_lidar = [file_names_lidar[_] for _ in range(len(num_lidar_points_list))\
                                            if num_lidar_points_list[_] >= N ]
    print(f'There are {len(file_names_lidar)} files greater than {N} points')
    
    # Save list to file
    save_list_to_pfile(file_names_lidar, lidar_file_list)

### LIDAR DATA PROCESSING

In [60]:
def get_image_files(lidar_file, method_type):
    # Create Lidar_x Lidar_y Lidar_z directory
    lx_file = extract_sensor_file_name(lidar_file, root_path, f'lidar-x-{method_type}', 'png')
    ly_file = extract_sensor_file_name(lidar_file, root_path, f'lidar-y-{method_type}', 'png')
    lz_file = extract_sensor_file_name(lidar_file, root_path, f'lidar-z-{method_type}', 'png')
    l_color_file = extract_sensor_file_name(lidar_file, root_path, 'lidar-image', 'png')
    img_file = extract_image_file_name_from_any_file_name(lidar_file, root_path)
 
    return img_file, lx_file, ly_file, lz_file, l_color_file

In [61]:
# Create Upsampled LIDAR image

# Iterate over Lidar Files
# for lidar_file in file_names_lidar:
def create_dense_lidar_images_upsample(lidar_file, project_lidar=False):
    if project_lidar:
        lidar_on_image, lidar_data  = get_lidar_on_image(lidar_file, config, root_path)
    else:
        lidar_data = np.load(lidar_file)
        
    ## CONSTANTS
    NEIGHBOUR_RADIUS = 40 #Pixels
    INVERSE_COFF = 0.5
    DEPTH_COFF = 0
    CUTOFF_THRESH = 0.4
    PIXEL_THRESH = 1/(1+INVERSE_COFF*NEIGHBOUR_RADIUS)
    
    lidar_on_image, lidar_data  = get_lidar_on_image(lidar_file, config, root_path)
    
    # Create Lidar_x Lidar_y Lidar_z directory
    img_file, lx_file, ly_file, lz_file, l_color_file = get_image_files(lidar_file, 'upsample')
    
    # TODO: Check if files already exist

    lx_cam_dir = get_prev_directory(lx_file)
    ly_cam_dir = get_prev_directory(ly_file)
    lz_cam_dir = get_prev_directory(lz_file)
    l_color_cam_dir = get_prev_directory(l_color_file)
    
    lx_dir = get_prev_directory(lx_cam_dir)
    ly_dir = get_prev_directory(ly_cam_dir)
    lz_dir = get_prev_directory(lz_cam_dir)
    l_color_dir = get_prev_directory(l_color_cam_dir)

    create_unique_dir(lx_dir)
    create_unique_dir(ly_dir)
    create_unique_dir(lz_dir)
    create_unique_dir(l_color_dir)
    
    create_unique_dir(lx_cam_dir)
    create_unique_dir(ly_cam_dir)
    create_unique_dir(lz_cam_dir)
    create_unique_dir(l_color_cam_dir)
    
    # Load Lidar Data and find max distance
    rows = (lidar_data['row'] + 0.5).astype(np.int)
    cols = (lidar_data['col'] + 0.5).astype(np.int)
    rows_float = np.array(lidar_data['row'])
    cols_float = np.array(lidar_data['col'])
    lidar_points = np.array(lidar_data['points'])
    lidar_depth = np.array(lidar_data['distance'])
    max_distance = np.max(lidar_depth)
    if DEBUG:
        print(f'max distance: {max_distance}')
    
    if DEBUG:
        print(f'Processing {lx_file}')
    
    # create X,Y,Z images
    img_file = extract_image_file_name_from_any_file_name(lidar_file, root_path)
    img_x = get_cv2_image(img_file ,cv2.COLOR_BGR2GRAY) # Grayscale image only has one channel
    img_dim = np.shape(img_x)

    img_x_num = np.zeros(img_dim)
    img_y_num = img_x_num.copy()
    img_z_num = img_x_num.copy()
    img_den = np.zeros(img_dim)
    
    x_or = np.zeros(img_dim)
    
    # Iterate Over LIDAR points
    if DEBUG:
        print(f'total Lidar Points: {len(rows)}')
    for lid_idx in range(len(rows)):
        idx_a = np.arange(np.maximum(rows[lid_idx] - NEIGHBOUR_RADIUS, 0),\
                      np.minimum(rows[lid_idx] + NEIGHBOUR_RADIUS + 1, img_dim[0]))
        idx_b = np.arange(np.maximum(cols[lid_idx] - NEIGHBOUR_RADIUS, 0),\
                      np.minimum(cols[lid_idx] + NEIGHBOUR_RADIUS + 1, img_dim[1]))
        
        dist_row = (rows_float[lid_idx] - idx_a)
        dist_col = (cols_float[lid_idx] - idx_b)
        
        if len(idx_a) != len(dist_row) or len(idx_b) != len(dist_col):
            print(str(rows_float[lid_idx]) + ", " + str(cols_float[lid_idx]))
            print(f'{len(idx_a)}, {len(idx_b)}')
            print(f'{len(dist_row)}, {len(dist_col)}')
            break
        
        dist_row_mat = np.array([dist_row]).T * np.ones(len(dist_col))
        dist_col_mat = np.ones((len(dist_row), 1)) * np.array([dist_col])

        temp_mat = ( 1 - DEPTH_COFF*lidar_depth[lid_idx]/max_distance)/\
                   ( 1 + INVERSE_COFF*np.sqrt( np.square(dist_row_mat) + np.square(dist_col_mat)))
        
        # Cap the lowest value of denominator 
        temp_mat[temp_mat < PIXEL_THRESH ] = 0.0
        
        img_den[np.ix_(idx_a,idx_b)] += temp_mat
        
#         img_den[np.ix_(idx_a,idx_b)] += ( 1 - DEPTH_COFF*lidar_data['distance'][lid_idx]/max_distance)/\
#                                    ( 1 + INVERSE_COFF*np.sqrt( np.square(dist_row_mat) + np.square(dist_col_mat)))
        
        img_x_num[np.ix_(idx_a,idx_b)] += img_den[idx_a][:,idx_b] * lidar_points[lid_idx,0]
        img_y_num[np.ix_(idx_a,idx_b)] += img_den[idx_a][:,idx_b] * lidar_points[lid_idx,1]
        img_z_num[np.ix_(idx_a,idx_b)] += img_den[idx_a][:,idx_b] * lidar_points[lid_idx,2]
    
    print(f'Creating Image: {lx_file}\n')
    
    # Cap the lowest value of denominator 
    img_den[img_den < CUTOFF_THRESH] = 0.0
    
    img_x_num = np.divide(img_x_num, img_den, out=np.zeros_like(img_x_num), where=img_den!=0) # Divide by 0 is a 0
    img_y_num = np.divide(img_y_num, img_den, out=np.zeros_like(img_y_num), where=img_den!=0) # Divide by 0 is a 0
    img_z_num = np.divide(img_z_num, img_den, out=np.zeros_like(img_z_num), where=img_den!=0) # Divide by 0 is a 0
    
    img_x_num = normalize_vector(img_x_num, 0.0, 2**16).astype(np.uint16)
    img_y_num = normalize_vector(img_y_num, 0.0, 2**16).astype(np.uint16)
    img_z_num = normalize_vector(img_z_num, 0.0, 2**16).astype(np.uint16)
    
#     img_x_num[np.argwhere(img_x_num == 0.0)] = 255.0
    
    cv2.imwrite(lx_file, img_x_num)
    cv2.imwrite(ly_file, img_y_num)
    cv2.imwrite(lz_file, img_z_num)
    if project_lidar:
        cv2.imwrite(l_color_file, lidar_on_image)
    
    if DEBUG:
        print(f'Saving {lx_file}')
        print(f'Saving {ly_file}')
        print(f'Saving {lz_file}')
        
    return img_file, lx_file, ly_file, lz_file, l_color_file

In [62]:
# Using Waslander code here
from ip_basic import depth_map_utils

def create_dense_lidar_images_ip_basic(lidar_file, project_lidar=False):
    if project_lidar:
        lidar_on_image, lidar_data  = get_lidar_on_image(lidar_file, config, root_path)
    else:
        lidar_data = np.load(lidar_file)
    
    # Create Lidar_x Lidar_y Lidar_z directory
    img_file, lx_file, ly_file, lz_file, l_color_file = get_image_files(lidar_file, 'ip')
    
    if False and Path(lx_file).is_file() and Path(ly_file).is_file() and Path(lz_file).is_file():
        return img_file, lx_file, ly_file, lz_file, l_color_file
    
    # TODO: Check if files already exist

    lx_cam_dir = get_prev_directory(lx_file)
    ly_cam_dir = get_prev_directory(ly_file)
    lz_cam_dir = get_prev_directory(lz_file)
    l_color_cam_dir = get_prev_directory(l_color_file)
    
    lx_dir = get_prev_directory(lx_cam_dir)
    ly_dir = get_prev_directory(ly_cam_dir)
    lz_dir = get_prev_directory(lz_cam_dir)
    l_color_dir = get_prev_directory(l_color_cam_dir)

    create_unique_dir(lx_dir)
    create_unique_dir(ly_dir)
    create_unique_dir(lz_dir)
    create_unique_dir(l_color_dir)
    
    create_unique_dir(lx_cam_dir)
    create_unique_dir(ly_cam_dir)
    create_unique_dir(lz_cam_dir)
    create_unique_dir(l_color_cam_dir)
    
    # Load Lidar Data and find max distance
    rows = (lidar_data['row'] + 0.5).astype(np.int)
    cols = (lidar_data['col'] + 0.5).astype(np.int)
    lidar_points = np.array(lidar_data['points'])
    lidar_points_x = normalize_vector(lidar_points[:,0], 2**8 * 0.1 + 1, 2**16 - 1)
    lidar_points_y = normalize_vector(lidar_points[:,1], 2**8 * 0.1 + 1, 2**16 - 1)
    lidar_points_z = normalize_vector(lidar_points[:,2], 2**8 * 0.1 + 1, 2**16 - 1)
    
    # create X,Y,Z images
    img_x = get_cv2_image(img_file ,cv2.COLOR_BGR2GRAY) # Grayscale image only has one channel
    img_dim = np.shape(img_x)

    img_x_num = np.zeros(img_dim, dtype=np.uint16)
    img_y_num = img_x_num.copy()
    img_z_num = img_x_num.copy()
    
    # Iterate Over LIDAR points
    if DEBUG:
        print(f'total Lidar Points: {len(rows)}')
        
    if DEBUG:
        print(f'Processing {lx_file}')
        
    for lid_idx in range(len(rows)):
        idx_a = np.clip(rows[lid_idx], 0, img_dim[0]-1)
        idx_b = np.clip(cols[lid_idx], 0, img_dim[1]-1)
        
        img_x_num[idx_a,idx_b] = lidar_points_x[lid_idx]
        img_y_num[idx_a,idx_b] = lidar_points_y[lid_idx]
        img_z_num[idx_a,idx_b] = lidar_points_z[lid_idx]
        
    projected_x = np.float32(img_x_num/256.0)
    projected_y = np.float32(img_y_num/256.0)
    projected_z = np.float32(img_z_num/256.0)

    projected_x = depth_map_utils.fill_in_fast( projected_x, max_depth=2**8 + 1)
    projected_y = depth_map_utils.fill_in_fast( projected_y, max_depth=2**8 + 1)
    projected_z = depth_map_utils.fill_in_fast( projected_z, max_depth=2**8 + 1)

    img_x_num = (projected_x * 256.0).astype(np.uint16)
    img_y_num = (projected_y * 256.0).astype(np.uint16)
    img_z_num = (projected_z * 256.0).astype(np.uint16)

    print(f'Creating Image: {lx_file}\n')

    cv2.imwrite(lx_file, img_x_num)
    cv2.imwrite(ly_file, img_y_num)
    cv2.imwrite(lz_file, img_z_num)
    if project_lidar:
        cv2.imwrite(l_color_file, lidar_on_image)
        
    return img_file, lx_file, ly_file, lz_file, l_color_file

In [63]:
# from multiprocessing import Pool
from multiprocessing import Pool

NUM_WORKERS = 6

def create_dense_lidar_images(file_names_lidar, image_range = (0, 100), use_mp=True,n_worker = NUM_WORKERS): 
    process_files = file_names_lidar[image_range[0]:image_range[1]]
    ip_files = []
    upsample_files = []
    
    if use_mp:
        pool1 = Pool(n_worker)
        start = time.time()
        pool1.map(create_dense_lidar_images_ip_basic, process_files)
        compute_time_ip = (time.time() - start)/num_images
        start = time.time()
        pool1.map(create_dense_lidar_images_upsample, process_files)
        compute_time_upsample = (time.time() - start)/num_images

    else:
        start = time.time()
        for lidar_file in process_files:
            out_ip = create_dense_lidar_images_ip_basic(lidar_file)
            ip_files.append(list(out_ip)[:-1])
        compute_time_ip = (time.time() - start)/num_images

        start = time.time()
        for lidar_file in process_files:
            out_upsample = create_dense_lidar_images_upsample(lidar_file)
            upsample_files.append(out_upsample)
        compute_time_upsample = (time.time() - start)/num_images

    print(f'Processing time per image (ip_basic): {compute_time_ip} seconds')
    print(f'Processing time per image (upsampling): {compute_time_upsample} seconds')
    
    return ip_files, upsample_files

In [64]:
# LIDAR data Processing
num_images = 20

ip_files = []
upsample_files = []
processed_lidar_files = file_names_lidar[0:num_images]
ip_files, upsample_files = \
         create_dense_lidar_images(file_names_lidar, image_range=(0,num_images), use_mp=True)

Creating Image: /hdd/a2d2-data/camera_lidar_semantic/20180807_145028/lidar-x-ip/cam_front_center/20180807145028_lidar-x-ip_frontcenter_000009806.png

Creating Image: /hdd/a2d2-data/camera_lidar_semantic/20180807_145028/lidar-x-ip/cam_front_center/20180807145028_lidar-x-ip_frontcenter_000009489.png

Creating Image: /hdd/a2d2-data/camera_lidar_semantic/20180807_145028/lidar-x-ip/cam_front_center/20180807145028_lidar-x-ip_frontcenter_000006176.png

Creating Image: /hdd/a2d2-data/camera_lidar_semantic/20180807_145028/lidar-x-ip/cam_front_center/20180807145028_lidar-x-ip_frontcenter_000009789.png

Creating Image: /hdd/a2d2-data/camera_lidar_semantic/20180807_145028/lidar-x-ip/cam_front_center/20180807145028_lidar-x-ip_frontcenter_000006128.png

Creating Image: /hdd/a2d2-data/camera_lidar_semantic/20180807_145028/lidar-x-ip/cam_front_center/20180807145028_lidar-x-ip_frontcenter_000009786.png

Creating Image: /hdd/a2d2-data/camera_lidar_semantic/20180807_145028/lidar-x-ip/cam_front_center/201

### Dataset saving

In [31]:
# Create a list of all input images
if not len(ip_files) or not len(upsample_files):
    for lidar_file in processed_lidar_files:
        if 'front' in lidar_file:
            out_ip = get_image_files(lidar_file, 'ip')
            out_upsample = get_image_files(lidar_file, 'upsample')
            ip_files.append(list(out_ip)[:-1])
            upsample_files.append(list(out_upsample)[:-1])

save_list_to_pfile(ip_files, root_path + '../dataset/ip_inputs.pkl')
save_list_to_pfile(upsample_files, root_path + '../dataset/upsample_inputs.pkl')

### Ground Truth Conditioning

In [32]:
def create_binary_gt(lidar_files, color=(255,0,255)):
    gt_files = []
    for lidar_file in lidar_files:
        if 'front' in lidar_file:
            assert Path(lidar_file).is_file(), f'{lidar_file} is not a file'
            # Get Label file
            label_file = extract_semantic_file_name_from_any_file_name(lidar_file, root_path)
            gt_file = extract_sensor_file_name(lidar_file, root_path, 'image-gt', 'png')

            # Skip if file exists
            if Path(gt_file).is_file():
                gt_files.append(gt_file)
                continue

            # Mask for color
            label_img = cv2.imread(str(label_file), 1)
            B = label_img[:,:,0] == color[0]
            G = label_img[:,:,1] == color[1]
            R = label_img[:,:,2] == color[2]
            road_area = B & G & R
            gt_img = road_area.astype(dtype=np.uint8) * 255

            # Create GT folder and save file
            if not Path(gt_file).is_file():
                gt_cam_dir = get_prev_directory(gt_file)
                gt_dir = get_prev_directory(gt_cam_dir)
                create_unique_dir(gt_dir)
                create_unique_dir(gt_cam_dir)
                cv2.imwrite(gt_file, gt_img)

            # Append file to list
            gt_files.append(gt_file)
        
    return gt_files

In [33]:
gt_files = create_binary_gt(processed_lidar_files)
save_list_to_pfile(gt_files, root_path + '../dataset/outputs.pkl')

In [34]:
len(gt_files)

3858

In [34]:
means_img = np.array([0.0, 0.0, 0.0])
means_lidar = np.array([0.0, 0.0, 0.0])

meanss_img = np.array([0.0, 0.0, 0.0])
meanss_lidar = np.array([0.0, 0.0, 0.0])

idx = 0

for ip in ip_files:
    if Path(ip[0]).is_file():
        idx = idx+1
        img = cv2.imread(str(ip[0]), cv2.COLOR_BGR2RGB)
        img_x = cv2.imread(str(ip[1]), 1)
        img_y = cv2.imread(str(ip[2]), 1)
        img_z = cv2.imread(str(ip[3]), 1)
        img_lidar = cv2.merge((img_x, img_y, img_z))
        
        means_img += np.array([(img[0]).mean(),(img[1]).mean(), (img[2]).mean()])
        means_lidar += np.array([(img_lidar[0]).mean(),(img_lidar[1]).mean(), (img_lidar[2]).mean()])
        
        img_s = img.astype(np.uint32)**2
        img_lidar_s = img_lidar.astype(np.uint32)**2
        meanss_img += np.array([(img_s[0]).mean(),(img_s[1]).mean(), (img_s[2]).mean()])
        meanss_lidar += np.array([(img_lidar_s[0]).mean(),(img_lidar_s[1]).mean(), (img_lidar_s[2]).mean()])
        
        print(f'Done with img{idx}')
        

Done with img1
Done with img2
Done with img3
Done with img4
Done with img5
Done with img6
Done with img7
Done with img8
Done with img9
Done with img10
Done with img11
Done with img12
Done with img13
Done with img14
Done with img15
Done with img16
Done with img17
Done with img18
Done with img19
Done with img20
Done with img21
Done with img22
Done with img23
Done with img24
Done with img25
Done with img26
Done with img27
Done with img28
Done with img29
Done with img30
Done with img31
Done with img32
Done with img33
Done with img34
Done with img35
Done with img36
Done with img37
Done with img38
Done with img39
Done with img40
Done with img41
Done with img42
Done with img43
Done with img44
Done with img45
Done with img46
Done with img47
Done with img48
Done with img49
Done with img50
Done with img51
Done with img52
Done with img53
Done with img54
Done with img55
Done with img56
Done with img57
Done with img58
Done with img59
Done with img60
Done with img61
Done with img62
Done with img63
D

Done with img490
Done with img491
Done with img492
Done with img493
Done with img494
Done with img495
Done with img496
Done with img497
Done with img498
Done with img499
Done with img500
Done with img501
Done with img502
Done with img503
Done with img504
Done with img505
Done with img506
Done with img507
Done with img508
Done with img509
Done with img510
Done with img511
Done with img512
Done with img513
Done with img514
Done with img515
Done with img516
Done with img517
Done with img518
Done with img519
Done with img520
Done with img521
Done with img522
Done with img523
Done with img524
Done with img525
Done with img526
Done with img527
Done with img528
Done with img529
Done with img530
Done with img531
Done with img532
Done with img533
Done with img534
Done with img535
Done with img536
Done with img537
Done with img538
Done with img539
Done with img540
Done with img541
Done with img542
Done with img543
Done with img544
Done with img545
Done with img546
Done with img547
Done with img5

Done with img972
Done with img973
Done with img974
Done with img975
Done with img976
Done with img977
Done with img978
Done with img979
Done with img980
Done with img981
Done with img982
Done with img983
Done with img984
Done with img985
Done with img986
Done with img987
Done with img988
Done with img989
Done with img990
Done with img991
Done with img992
Done with img993
Done with img994
Done with img995
Done with img996
Done with img997
Done with img998
Done with img999
Done with img1000
Done with img1001
Done with img1002
Done with img1003
Done with img1004
Done with img1005
Done with img1006
Done with img1007
Done with img1008
Done with img1009
Done with img1010
Done with img1011
Done with img1012
Done with img1013
Done with img1014
Done with img1015
Done with img1016
Done with img1017
Done with img1018
Done with img1019
Done with img1020
Done with img1021
Done with img1022
Done with img1023
Done with img1024
Done with img1025
Done with img1026
Done with img1027
Done with img1028
Do

KeyboardInterrupt: 

In [79]:
std_img = np.sqrt(meanss_img/idx - (means_img/idx)**2)
std_lidar = np.sqrt(meanss_lidar/idx - (means_lidar/idx)**2)
mean_img = means_img/idx
mean_lidar = means_lidar/idx

In [80]:
mean_img/255, std_img/255

(array([0.63263481, 0.63265741, 0.62899464]),
 array([0.25661512, 0.25698695, 0.2594808 ]))

In [81]:
mean_lidar/255, std_lidar/255

(array([0.16190466, 0.16355008, 0.16611974]),
 array([0.25623018, 0.256974  , 0.25811531]))